In [ ]:
import os
from pathlib import Path
import pandas as pd
import tsfel


ACTIVITIES = ['LAYING', 'SITTING', 'STANDING', 'WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS']
SLICE = slice(100, 600)  # same as your code
FS = 50                  # sampling rate

cfg = tsfel.get_features_by_domain()
for domain in cfg:
    for feature in cfg[domain]:
        cfg[domain][feature]['use'] = 'yes'  # use all features

def store_tsfel_split(split_name: str):
    """
    read Combined/<split>/<activity>/*.csv,
    extract TSFEL features, and store them under
    Datasets/TSFEL_3axes_allfeatures/<split>/<activity>/*.csv
    """
    base_dir = os.path.join('Datasets', 'Combined', split_name)
    output_base_dir = os.path.join('Datasets', 'TSFEL_3axes_allfeatures', split_name)

    for activity in ACTIVITIES:
        activity_dir = os.path.join(base_dir, activity)
        output_activity_dir = os.path.join(output_base_dir, activity)
        Path(output_activity_dir).mkdir(parents=True, exist_ok=True)

        if not os.path.isdir(activity_dir):
            print(f"[WARN] Missing folder: {activity_dir}")
            continue

        subject_files = [f for f in os.listdir(activity_dir) if f.lower().endswith('.csv')]
        if not subject_files:
            print(f"[WARN] No CSV files in: {activity_dir}")
            continue

        print(f"[{split_name}] {activity}: {len(subject_files)} files")
        for file in subject_files:
            try:
                file_path = os.path.join(activity_dir, file)
                df = pd.read_csv(file_path).iloc[SLICE, :]   

                # Your TSFEL extraction call (all features enabled)
                features = tsfel.time_series_features_extractor(cfg, df, fs=FS, verbose=0)

                subject_id = os.path.splitext(file)[0]
                output_file = os.path.join(output_activity_dir, f'{subject_id}.csv')
                features.to_csv(output_file, index=False)
            except Exception as e:
                print(f"[ERROR] {split_name}/{activity}/{file}: {e}")


store_tsfel_split('Train')
store_tsfel_split('Test')

print("\n[Done] Stored TSFEL features at:")
print(os.path.join('Datasets', 'TSFEL_3axes_allfeatures'))
